# Numeric Results Summary
The purpose of this file is to summarize and print all the numeric results announced in Mahdavi et al. (2021) Environmental Pollution (Airborne PM Phase).

In [60]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, ranksums
exec(open(r'C:\PhD Research\Generic Codes\notion_corrections.py').read())

In [61]:
### Weekly runtime fractions

rt = pd.read_excel(backslash_correct(r'C:\PhD Research\Airborne\Processed\runtime_sum.xlsx'))
rt = rt[rt['Mode'] == "On"]['runtime']

### PM and runtime correlation results
pm = pd.read_excel(backslash_correct(r'C:\PhD Research\Airborne\Processed\pm_master_all.xlsx'))
pm.columns

['visit', 'TSP Concentration', 'TSP Concentration Error', 'PM10 SCI',
       'PM10 SCI Error', 'PM2.5 SCI', 'PM2.5 SCI Error', 'PM1 SCI',
       'PM1 SCI Error', 'PM2.5 PEM', 'PM2.5 PEM Error']

a = spearmanr(pm['TSP Concentration'], rt)
b = spearmanr(pm['PM10 SCI'], rt)
c = spearmanr(pm['PM2.5 SCI'], rt)
d = spearmanr(pm['PM1 SCI'], rt)

### DustTrak and runtime correlation results
rtm = pd.read_excel(backslash_correct(r'C:\PhD Research\Airborne\Processed\runtime_master.xlsx'))
dt = pd.read_excel(backslash_correct(r'C:\PhD Research\Airborne\Processed\\') + 'dt_drx.xlsx')
dt.rename(columns = {'Date': 'Time'}, inplace = True)

df = dt.merge(rtm, on = 'Time', how = 'outer')
df.drop(['Time', 'Pressure', 'visit_y'], axis = 1, inplace = True)
df.rename(columns = {'visit_x': 'visit'}, inplace = True)
df = df[df['Mode'] != 'Transient']
df.dropna(subset = ['PM1'], inplace = True)

df.head()

e = ranksums(df[df['Mode'] == 'Off']['PM2.5'], df[df['Mode'] != 'Off']['PM2.5'])
e2 = ranksums(df[df['Mode'] == 'Compressor']['PM2.5'], df[df['Mode'] == 'Fan Only']['PM2.5'])

### DC1700 and runtime correlation results
cor_f = 6.7223 / 17.9024 # DusTrak correction factor for concentration  adjustment (ratio of gracimetric concnetration to laser recorded concentrations)

df2 = pd.read_excel(backslash_correct(r'C:\PhD Research\Airborne\Processed\\') + 'dc_1700.xlsx')
df2.rename(columns = {'Date/Time': 'Time'}, inplace = True)

rt2 = pd.read_excel(backslash_correct(r'C:\PhD Research\Airborne\Processed\runtime_master.xlsx'))
rt2.drop(['visit'], axis = 1, inplace = True)

df2 = df2.merge(rt2, on = 'Time', how = 'inner')
df2.columns
for col in df2.columns:
    df2.rename(columns = {col : col.strip()}, inplace = True)
  
df2['Concentration'] = (df2['Small'] - df2['Large']) * 35.3147 * cor_f
df2.drop(['Pressure', 'Small', 'Large'], axis = 1, inplace = True)

df2['Mode'] = np.where(df2['Mode'] == 'Off', 'Off', 'On')

f = ranksums(df2[df2['Mode'] == 'Off']['Concentration'], df2[df2['Mode'] != 'Off']['Concentration'])

### Other descriptive statistic for PM
g = pm['PM2.5 SCI'].describe()
g2 = pm['TSP Concentration'].describe()

In [62]:
### Descriptive Statistics and Numerical Results (in the PM Phase of the paper)

print('Descriptive Statistics and Numeric Results:\n')

print('Runtime fractions were measured as {}, {}, {}, {}, {}, and {}%, respectively for the six weeks of the study.'.format(round(rt.iloc[0],1),
                                                                                                                         round(rt.iloc[1],1),
                                                                                                                         round(rt.iloc[2],1),
                                                                                                                         round(rt.iloc[3],1),
                                                                                                                         int(round(rt.iloc[4],0)),
                                                                                                                         int(round(rt.iloc[5],0))))

print("The Spearman's rank correlation p-value between TSP concentration and runtime fraction was {}.".format(round(a[1],2)))
print("The Spearman's rank correlation p-value between PM10 and runtime fraction was {}.".format(round(b[1],2)))
print("The Spearman's rank correlation p-value between PM2.5 and runtime fraction was {}.".format(round(c[1],2)))
print("The Spearman's rank correlation p-value between PM1 and runtime fraction was {}.".format(round(d[1],2)))

print('The ranksum p-value result for the comparison of PM2.5 at the off and running system cases is {}.'.format(round(e[1],4)))
print('The ranksum p-value result for the comparison of PM2.5 at the compressor and fan only cases is {}.'.format(round(e2[1],4)))

print('The ranksum p-value result for the comparison of numer concentration at the off and running system cases is {}.'.format(round(f[1],4)))

print('PM2.5 concentration ranged from {} to {} with a median of {} μg/m3.'.format(round(g[3],1),
                                                                                  round(g[7],1),
                                                                                  round(g[5],1)))

print('TSP concentration ranged from {} to {} with a median of {} μg/m3.'.format(round(g2[3],1),
                                                                                  round(g2[7],1),
                                                                                  round(g2[5],1)))


Descriptive Statistics and Numeric Results:

Runtime fractions were measured as 5.8, 10.0, 18.8, 6.6, 100, and 100%, respectively for the six weeks of the study.
The Spearman's rank correlation p-value between TSP concentration and runtime fraction was 0.33.
The Spearman's rank correlation p-value between PM10 and runtime fraction was 0.21.
The Spearman's rank correlation p-value between PM2.5 and runtime fraction was 0.4.
The Spearman's rank correlation p-value between PM1 and runtime fraction was 0.33.
The ranksum p-value result for the comparison of PM2.5 at the off and running system cases is 0.0.
The ranksum p-value result for the comparison of PM2.5 at the compressor and fan only cases is 0.0.
The ranksum p-value result for the comparison of numer concentration at the off and running system cases is 0.004.
PM2.5 concentration ranged from 3.0 to 9.2 with a median of 7.2 μg/m3.
TSP concentration ranged from 8.9 to 34.4 with a median of 17.4 μg/m3.


End of Code